<a href="https://colab.research.google.com/github/AhmedAlshoki/InceptionModule/blob/main/Inception_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install bayesian-optimization

In [ ]:
import cv2 as cv
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from tensorflow.keras import layers , regularizers , activations
from bayes_opt import BayesianOptimization
import numpy as np
import tensorflow.keras.initializers
import statistics
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras as keras
from tensorflow.keras import optimizers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
def Define_Data():
    X = []
    Y = []

    myfolder = '/content/drive/MyDrive/Graduation Project/Data_Augmented/'
    My_Data= os.listdir(myfolder)
    index = 0
    for folder in My_Data:
        s=r'/content/drive/MyDrive/Graduation Project/Data_Augmented/'+folder
        for img in os.listdir(s):
            image = cv.imread(os.path.join(s,img))
            
            if image is not None:
                X.append(image)
                Y.append(index)
        index +=1
        print(len(X))
    
    X_Train , X_Test , Y_Train , Y_Test = train_test_split(X,Y,test_size=0.25,shuffle=True) 
    X_Test = np.array(X_Test).reshape(len(X_Test),150,150,3)
    X_Train=np.array(X_Train).reshape(len(X_Train),150,150,3)
    Y_Train= np.array(Y_Train)
    Y_Test = np.array(Y_Test)

    #One Hit Encoding 
    onehot_encoder = OneHotEncoder(sparse=False)
    Y_Train = Y_Train.reshape(len(Y_Train), 1)
    Y_Test = Y_Test.reshape(len(Y_Test), 1)
    
    Y_Train = onehot_encoder.fit_transform(Y_Train)
    Y_Test = onehot_encoder.fit_transform(Y_Test)
    #scale
    X_Train = X_Train.astype("float32") / 255
    X_Test = X_Test.astype("float32") / 255
    X_Train = tf.convert_to_tensor(X_Train, dtype=tf.float32)
    X_Test = tf.convert_to_tensor(X_Test, dtype=tf.float32)
    Y_Train = tf.convert_to_tensor(Y_Train, dtype=tf.float32)
    Y_Test = tf.convert_to_tensor(Y_Test, dtype=tf.float32)
    return X_Train , X_Test , Y_Train , Y_Test
X_Train , X_Test , Y_Train , Y_Test = Define_Data()  

In [ ]:
def inception_module(previous_layer,num_of_filters):
    k = keras.backend.set_image_data_format('channels_last')
    activation = activations.relu
    #first layer 1*1
    layer_1 = layers.Conv2D(num_of_filters[0],(1,1),padding = 'same',activation=activation)(previous_layer)
    #second layer 3*3
    layer_2 = layers.Conv2D(num_of_filters[1],(1,1),padding = 'same',activation=activation)(previous_layer)
    layer_2_1 = layers.Conv2D(num_of_filters[2],(3,3),padding = 'same',activation=activation)(layer_2)
    #third_layer Max Pooling
    layer_3 = layers.MaxPool2D((2,2),strides= (1,1),padding='same')(previous_layer)
    layer_3_1 = layers.Conv2D(num_of_filters[3],(1,1),padding='same',activation=activation)(layer_3)
    #fourth layer 5*5
    layer_4 = layers.Conv2D(num_of_filters[4],(1,1),padding = 'same',activation=activation)(previous_layer)
    layer_4_1 = layers.Conv2D(num_of_filters[5],(5,5),padding = 'same',activation=activation)(layer_4)

    #concatenate them all
    final_layer = layers.concatenate([layer_1,layer_2_1,layer_3_1,layer_4_1], axis=3)
    return final_layer

In [ ]:
def generate_model(number_of_inception_layers,drop_out):
   activation = activations.relu
   numper_of_filters_for_first_layer= [64,96,128,32,16,32]
   Input_layer = layers.Input(shape=(150, 150, 3))
   layer = layers.Conv2D(64, (7, 7), strides=1,padding='same', activation=activation)(Input_layer)
   layer = layers.BatchNormalization()(layer)
   MaxPooling = layers.MaxPool2D((2, 2), strides=2,padding='same')(layer)
   layer = layers.Conv2D(192, (3, 3), strides=1,padding='same', activation=activation)(MaxPooling)
   layer = layers.BatchNormalization()(layer)
   layer = layers.MaxPool2D((3, 3), strides=2,padding='same')(layer)
   for i in range(int(number_of_inception_layers)):
       layer = inception_module(layer,numper_of_filters_for_first_layer)
       layer = layers.BatchNormalization()(layer)
       if i %2 == 0:
           layer = inception_module(layer,numper_of_filters_for_first_layer)
           layer = layers.BatchNormalization()(layer)
           layer = layers.MaxPool2D((3, 3), strides=2,padding='same')(layer)
           numper_of_filters_for_first_layer *=2
       
   Drop = layers.Dropout(drop_out)(layer)
   Flat_layer = layers.Flatten()(Drop)
   Hidden = layers.Dense(10, activation='relu',name="First_Hidden")(Flat_layer)
   Hidden = layers.Dense(3, activation='relu',name="Second_Hidden")(Hidden)
   #Hidden = layers.Dense(10, activation='relu',name="Last_Hidden")(Hidden)
   Output_layer = layers.Dense(units=3, activation='softmax',name="Final_Layer_1") (Hidden)
   model = keras.Model(inputs =Input_layer,outputs = Output_layer )
   return model

In [ ]:
def evaluate_the_model(number_of_inception_layers,drop_out,learning_rate):
    model = generate_model(number_of_inception_layers,drop_out)
    model.compile(loss= 'categorical_crossentropy', optimizer=optimizers.SGD(lr=learning_rate),metrics='accuracy')
    monitor = EarlyStopping(monitor='accuracy',patience=50, mode='min', restore_best_weights=True)
    model.fit(X_Train,Y_Train,callbacks=[monitor],epochs=280,verbose=0)
    score = model.evaluate(X_Test, Y_Test)
    tensorflow.keras.backend.clear_session()
    return score[1]

In [ ]:
pbounds = {'number_of_inception_layers' : (3 , 8),
           'learning_rate': (0.0, 1.0),
           'drop_out': (0.0, 0.499)
          }
optimizer = BayesianOptimization(f=evaluate_the_model,pbounds=pbounds,verbose=10)
optimizer.maximize(init_points=10, n_iter=100,)
print(optimizer.max)